# Quick Guide

In [1]:
# PUBLIC API
# get_ticker(tickers)
# get_trades(tickers)
# get_depth(tickers)

# PRIVATE API
# account = vipbtc.TradeAPI(key, secret)
# account.getInfo()                            
# account.transHistory()                       
# account.trade(ticker,ttype, amount, price)          
# account.tradeHistory()                       
# account.openOrders()                         
# account.cancelOrder(ticker,ttype, order_id)

# Self Define 
# buy((ticker,amount,price,target,stop_loss)
# sell(ticker,amount,price)
# get_balance()
# generate_wallet()

# Import Modules

In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import math
import datetime as dt
import json
import time
import sys
import vipbtc
import vipbtc_btc
from IPython.display import clear_output

# Available API and Currencies

In [3]:
API=dict()
API["ticker"] = "https://vip.bitcoin.co.id/api/{}/ticker"
API["trades"] = "https://vip.bitcoin.co.id/api/{}/trades" 
API["depth"]  = "https://vip.bitcoin.co.id/api/{}/depth" 

Tickers=dict()
Tickers["Bitcoin"]  = "btc_idr"
Tickers["Dogecoin"] = "doge_btc"
Tickers["DASH"]     = "drk_btc"
Tickers["BitShares"]= "bts_btc"
Tickers["NEM"]      = "nem_btc"

Currency=dict()
Currency["Bitcoin"]  = "btc"
Currency["Dogecoin"] = "doge"
Currency["DASH"]     = "drk"
Currency["BitShares"]= "bts"
Currency["NEM"]      = "nem"


# Public API Wrapper

In [4]:
def get_ticker(currency):
    request=API["ticker"].format(Tickers[currency])
    raw_result=requests.get(request)
    result=pd.DataFrame.from_dict(raw_result.json(),orient="index")
    final_result=result.copy(deep=True)
    return final_result

def get_trades(currency):
    request=API["trades"].format(Tickers[currency])
    raw_result=requests.get(request)
    result=pd.DataFrame.from_dict(raw_result.json(),orient="columns")
    final_result=result.copy(deep=True)
    return final_result

def get_depth(currency):
    request=API["depth"].format(Tickers[currency])
    raw_result=requests.get(request)
    raw_result=raw_result.json()
    lowest_length=min(len(raw_result["buy"]),len(raw_result["sell"]))
    raw_result["buy"]=raw_result["buy"][:lowest_length]
    raw_result["sell"]=raw_result["sell"][:lowest_length]
    result=pd.DataFrame.from_dict(raw_result,orient="columns")
    
    result["buy_price"]=np.nan
    result["buy_amount"]=np.nan
    result["sell_price"]=np.nan
    result["sell_amount"]=np.nan
    
    for i in result.index:
        result.set_value(i,"buy_price",result["buy"][i][0])
        result.set_value(i,"buy_amount",result["buy"][i][1])
        result.set_value(i,"sell_price",result["sell"][i][0])
        result.set_value(i,"sell_amount",result["sell"][i][1])
    
    final_result=result.drop(["buy","sell"],axis=1)
    return final_result

# Analyzer Functions

In [5]:
def initial_buy(amount_idr):
    currency="Bitcoin"
    best_price=get_currency_sell_price(currency)
    account.trade(Tickers[currency],Currency[currency],"buy", amount_idr, best_price)

def final_sell():
    currency="Bitcoin"
    balance=get_balance()["Balance"]["btc"]
    best_price=get_currency_sell_price(currency)
    account.trade(Tickers[currency],Currency[currency],"sell", balance, best_price)   
    
def buy(currency,target_pct,stop_loss_pct,wallet):
    balance=get_balance()["Balance"]["btc"]
    best_price=get_currency_sell_price(currency)
    account_btc.trade(Tickers[currency],Currency[currency],"buy", balance, best_price)
    wallet.set_value(currency,"Buy Price",best_price)
    wallet.set_value(currency,"Target",best_price*(1+(target_pct/100)))
    wallet.set_value(currency,"Stop Loss",best_price*(1-(stop_loss_pct/100)))
    return wallet
    
def sell(currency,wallet):
    balance=get_balance()["Balance"][Currency[currency]]
    best_price=get_currency_buy_price(currency)
    account_btc.trade(Tickers[currency],Currency[currency],"sell", balance, best_price )   
    wallet["Buy Price"][currency]=0
    wallet["Target"][currency]=0
    wallet["Stop Loss"][currency]=0
    return wallet

def get_balance():
    b=pd.DataFrame.from_dict(list(account.getInfo()["return"]["balance"].items()))
    b.index=list(account.getInfo()["return"]["balance"])
    b.drop([0],axis=1,inplace=True)
    b.columns=["Balance"]
    
    b_h=pd.DataFrame.from_dict(list(account.getInfo()["return"]["balance_hold"].items()))
    b_h.index=list(account.getInfo()["return"]["balance_hold"])
    b_h.drop([0],axis=1,inplace=True)
    b_h.columns=["Balance Hold"]
    
    return pd.concat([b,b_h],axis=1)

def profit(initial_balance):
    total_profit=[]
    
def get_price():
    prices={}
    data=pd.DataFrame(index=list(Currency),columns=["Price"])
    for c in list(Currency):
        data.set_value(c,"Price",float(get_ticker(c)["sell"]["ticker"]))
    return data

def get_currency_sell_price(currency):
    return get_depth(currency)["sell_price"][0]

def get_currency_buy_price(currency):
    return get_depth(currency)["buy_price"][0]

# Other Functions

In [6]:
def generate_wallet():
    wallet=pd.DataFrame(index=list(Currency),columns=["Stop Loss","Buy Price","Target"]) 
    wallet.fillna(0,inplace=True)
    return wallet

# Private API

In [7]:
# AVAILABLE METHOD
# account = vipbtc.TradeAPI(key, secret)
# account.getInfo()                            
# account.transHistory()                       
# account.trade(ticker,ttype, amount, price)          
# account.tradeHistory()                       
# account.openOrders()                         
# account.cancelOrder(ticker,ttype, order_id)

# Identity
key = ""
secret = ""

# Initialization

In [8]:
account     = vipbtc.TradeAPI(key, secret)
account_btc = vipbtc_btc.TradeAPI_btc(key, secret)
wallet  = generate_wallet()

# Trading Bot

In [9]:
# Initialization
account     = vipbtc.TradeAPI(key, secret)
account_btc = vipbtc_btc.TradeAPI_btc(key, secret)
wallet  = generate_wallet()
target_currency = "Dogecoin"
target          = 0.75
stop_loss       = 0.5
hi_threshold    = 0.2
lo_threshold    = 0.2
investment=60000

In [ ]:
clear_output()
print("========================  STATUS  ========================")
print("  Initializing Bot")
print("==========================================================")
            
time.sleep(60)

clear_output()
print("========================  STATUS  ========================")
print("  Starting Bot")
print("==========================================================")

reference_price=get_currency_sell_price(target_currency)
buyed=True
sold_count=0
counter=-1
target_reached=0
stop_loss_reached=0
high_threshold=0
low_threshold=0
starting_time=dt.datetime.now()

initial_buy(investment)

wallet=buy(target_currency,target,stop_loss,wallet)

while(True):
    counter+=1
    if not(buyed):
        if(get_currency_sell_price(target_currency)<=reference_price*(1-(lo_threshold/100))):
            wallet=buy(target_currency,target,stop_loss,wallet)
            current_time=dt.datetime.now()
            dur=str(current_time-starting_time).split(":")
            low_threshold+=1
            clear_output()
            print("========================  STATUS  ========================")
            print("  BUYED")
            print("  Low Threshold Reached")
            print()
            print("  Target Reached          : {}".format(target_reached))
            print("  Stop Loss Reached       : {}".format(stop_loss_reached))
            print()
            print("  High Threshold Reached  : {}".format(high_threshold))
            print("  Low Threshold Reached   : {}".format(low_threshold))
            print()                  
            print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
            print("==========================================================")
            print()
            buyed=True
        
        elif(get_currency_sell_price(target_currency)>=reference_price*(1+(hi_threshold/100))):
            wallet=buy(target_currency,target,stop_loss,wallet)
            current_time=dt.datetime.now()
            dur=str(current_time-starting_time).split(":")
            high_threshold+=1
            clear_output()
            print("========================  STATUS  ========================")
            print("  BUYED")
            print("  High Threshold Reached")
            print()
            print("  Target Reached          : {}".format(target_reached))
            print("  Stop Loss Reached       : {}".format(stop_loss_reached))
            print()
            print("  High Threshold Reached  : {}".format(high_threshold))
            print("  Low Threshold Reached   : {}".format(low_threshold))
            print()
            print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
            print("==========================================================")
            print()
            buyed=True
            
    if(buyed):
        while(True):    
            counter+=1
            if(get_currency_buy_price(target_currency)>=wallet["Target"][target_currency]):
                wallet=sell(target_currency,wallet)
                reference_price=get_currency_sell_price(target_currency)
                current_time=dt.datetime.now()
                dur=str(current_time-starting_time).split(":")
                clear_output()
                target_reached+=1
                buyed=False
                sold_count+=1
                print("========================  STATUS  ========================")
                print("  SOLD")
                print("  Target Price Reached")
                print()
                print("  Target Reached          : {}".format(target_reached))
                print("  Stop Loss Reached       : {}".format(stop_loss_reached))
                print()
                print("  High Threshold Reached  : {}".format(high_threshold))
                print("  Low Threshold Reached   : {}".format(low_threshold))
                print()
                print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
                print("==========================================================")
                print()

                break
                
            elif(get_currency_buy_price(target_currency)<=wallet["Stop Loss"][target_currency]):
                wallet=sell(target_currency,wallet)
                reference_price=get_currency_sell_price(target_currency)
                current_time=dt.datetime.now()
                dur=str(current_time-starting_time).split(":")
                clear_output()
                stop_loss_reached+=1
                buyed=False
                sold_count+=1
                
                print("========================  STATUS  ========================")
                print("  SOLD")
                print("  Stop Loss Price Reached")
                print()
                print("  Target Reached          : {}".format(target_reached))
                print("  Stop Loss Reached       : {}".format(stop_loss_reached))
                print()
                print("  High Threshold Reached  : {}".format(high_threshold))
                print("  Low Threshold Reached   : {}".format(low_threshold))
                print()
                print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
                print("==========================================================")
                print()

                break
            
            if(counter%7==0):
                current_time=dt.datetime.now()
                dur=str(current_time-starting_time).split(":")
                clear_output()
                print("========================  STATUS  ========================")
                print("  Buy Price               : Rp {}".format(wallet["Buy Price"][target_currency]))
                print()
                print("  Target                  : Rp {}".format(wallet["Target"][target_currency]))
                print("  Current Price           : Rp {}".format(get_currency_buy_price(target_currency)))
                print("  Stop Loss               : Rp {}".format(wallet["Stop Loss"][target_currency]))
                print()
                print("  Target Reached          : {}".format(target_reached))
                print("  Stop Loss Reached       : {}".format(stop_loss_reached))
                print()
                print("  High Threshold Reached  : {}".format(high_threshold))
                print("  Low Threshold Reached   : {}".format(low_threshold))
                print()
                print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
                print("  Waiting For Sell Signal")
                print("==========================================================")
                print()
                counter=0
            time.sleep(3)
    
    if(counter%7==0):
        current_time=dt.datetime.now()
        dur=str(current_time-starting_time).split(":")
        clear_output()
        print("========================  STATUS  ========================")
        print("  High Threshold          : Rp {}".format(reference_price*1.005))
        print("  Current Price           : Rp {}".format(get_currency_sell_price(target_currency)))
        print("  Low Threshold           : Rp {}".format(reference_price*0.995))
        print()
        print("  Target Reached          : {}".format(target_reached))
        print("  Stop Loss Reached       : {}".format(stop_loss_reached))
        print()
        print("  High Threshold Reached  : {}".format(high_threshold))
        print("  Low Threshold Reached   : {}".format(low_threshold))
        print()
        print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
        print("  Waiting For Buy Signal")
        print("==========================================================")
        print()
        counter=0
    
    if(target_reached>=5 or stop_loss_reached>=5):
        break
        
    time.sleep(3)

final_sell()
current_time=dt.datetime.now()
dur=str(current_time-starting_time).split(":")
clear_output()
print("===========================  STATUS  =====================")
print("  ENDED")
print("  Target Reached          : {}".format(target_reached))
print("  Stop Loss Reached       : {}".format(stop_loss_reached))
print()
print("  High Threshold Reached  : {}".format(high_threshold))
print("  Low Threshold Reached   : {}".format(low_threshold))
print()
print("  Duration                : {} Hours {} Minutes {} Seconds".format(dur[0],dur[1],dur[2][:2]))
print("==========================================================")
print()